In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn import svm

In [ ]:
crypto_btc = yf.download(tickers='BTC-USD', start='2017-12-01', end='2022-12-31', interval='1d').drop(columns=['Adj Close']).round(2)

In [ ]:
crypto_btc['Percent Change'] = crypto_btc['Close'].pct_change()
crypto_btc = crypto_btc.dropna()

In [ ]:
crypto_btc['ema_9'] = crypto_btc['Close'].ewm(9).mean()

In [ ]:
crypto_btc['sma_5'] = crypto_btc['Close'].rolling(5).mean()

In [ ]:
crypto_btc['sma_10'] = crypto_btc['Close'].rolling(10).mean()

In [ ]:
crypto_btc['sma_15'] = crypto_btc['Close'].rolling(10).mean()

In [ ]:
crypto_btc['sma_30'] = crypto_btc['Close'].rolling(10).mean()

In [ ]:
crypto_btc = crypto_btc.dropna()
crypto_btc

In [ ]:
X = crypto_btc[['Open', 'High', 'Low', 'Volume', 'Percent Change', 'sma_5', 'sma_10', 'sma_15', 'sma_30', 'ema_9']].copy()
display(X.head())

In [ ]:
y = crypto_btc['Close']

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=6)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
X_train.head()

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Display sample data
X_test.head()

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)

In [ ]:
kr.fit(X_train, y_train)

In [ ]:
predict = kr.predict(X_train)
predict

In [ ]:
#btc_mae = ()
#btc_mae = mean_absolute_error(y, predict)
#print(btc_mae)

In [ ]:
#crypto_btc_pred = crypto_btc.copy().drop(columns=['Open','Volume','High','Low','Percent Change','sma_5', 'sma_10', 'sma_15', 'sma_30', 'ema_9']).rename(columns={'Close':'Actual Price'})
#crypto_btc_pred

#crypto_btc_pred['Predicted Price'] = predict
#crypto_btc_pred['Price Accuracy']= (crypto_btc_pred['Predicted Price']/crypto_btc_pred['Actual Price']-1)*100

In [ ]:
#crypto_btc['EMA_9'] = crypto_btc['Close'].ewm(9).mean().shift()
#crypto_btc['SMA_5'] = crypto_btc['Close'].rolling(5).mean().shift()
#crypto_btc['SMA_10'] = crypto_btc['Close'].rolling(10).mean().shift()
#crypto_btc['SMA_15'] = crypto_btc['Close'].rolling(15).mean().shift()
#crypto_btc['SMA_30'] = crypto_btc['Close'].rolling(30).mean().shift()
#crypto_btc['Percent Change'] = crypto_btc['Close'].pct_change()

In [ ]:
#crypto_btc = crypto_btc.dropna()
#crypto_btc

In [ ]:
crypto_btc.reset_index(inplace=True)
fig = px.line(crypto_btc, x='Date', y='Close', title='Close Price with Range Slider')
fig.update_layout(hovermode="x")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="2y", step="year", stepmode="backward"),
            dict(step="all")
            
        ])
    )
)
fig.show()

In [ ]:
'''
df = ()
def kernel_predict(df_pred):
    X = df.drop(columns=['Date'])
    y = df[['Close']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
    if df is crypto_btc:
        kr = KernelRidge(kernel='linear', alpha=5000)
    else:
        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)
    kr.fit(X_train, y_train)
    predict = kr.predict(X)
    if df is crypto_btc:
        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
    else:
        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})
    df_pred['Predicted Price'] = predict
    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100
    return df_pred
'''

In [ ]:
X = crypto_btc[['ema_9','sma_5']].shift().dropna().copy()

In [ ]:
crypto_btc

In [ ]:
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
crypto_btc_pred = crypto_btc.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
crypto_btc_pred['Predicted Price'] = predict
crypto_btc_pred['Price Accuracy']= (crypto_btc_pred['Predicted Price']/crypto_btc_pred['Actual Price']-1)*100
    

In [ ]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

In [ ]:
crypto_btc_pred

In [ ]:
crypto_btc_pred['Price Accuracy'].describe()

In [ ]:
crypto_btc_pred.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)

In [ ]:
print(f"BTC Kernel Ridge RMSE is {btc_rmse.round(2)}")

In [ ]:
future_df = pd.DataFrame()
  

future_df['Date'] = pd.date_range('2018-02-01', periods = 1826, freq ='1D')
future_df[['Close','Open','Volume']]= (0)

future_df


In [ ]:
crypto_btc
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
btc_future = future_df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
btc_future['Predicted Price'] = predict

btc_future 

In [ ]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

In [ ]:
crypto_btc2 = yf.download(tickers='BTC-USD', start='2018-01-01', end='2023-01-30').drop(columns=['Adj Close']).round(2).reset_index()
crypto_btc2['EMA_9'] = crypto_btc2['Close'].ewm(9).mean().shift()
crypto_btc2['SMA_5'] = crypto_btc2['Close'].rolling(5).mean().shift()
crypto_btc2['SMA_10'] = crypto_btc2['Close'].rolling(10).mean().shift()
crypto_btc2['SMA_15'] = crypto_btc2['Close'].rolling(15).mean().shift()
crypto_btc2['SMA_30'] = crypto_btc2['Close'].rolling(30).mean().shift()
crypto_btc2['Percent Change'] = crypto_btc2['Close'].pct_change()
#crypto_btc2 = crypto_btc2.dropna()
crypto_btc2
btc_future['Actual Price'] = crypto_btc2['Close'].round(0)
btc_future['Price Accuracy']= (btc_future['Predicted Price']/btc_future['Actual Price']-1)*100
btc_future.set_index('Date').tail(30)

In [ ]:
display(btc_future.loc[btc_future['Date'] == '2020-01-27'])
display(btc_future.loc[btc_future['Date'] == '2020-02-27'])

In [ ]:
from bokeh.models.formatters import BasicTickFormatter

btc_future.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)